# Get started with authentication and packages

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!earthengine authenticate
import ee
ee.Initialize()

In [ ]:
# Download .txt file(s) for selected NLDAS file names (based on variable names and time period) from https://disc.sci.gsfc.nasa.gov/datasets?keywords=NLDAS&page=1
# Save the .txt file(s) to Cloud bucket that is connected to the Google Colab account
!gsutil cp gs://rangelands/Moisture/*txt .

In [ ]:
import re
import json
import glob
import os
import PIL
import pandas as pd
import numpy as np

In [ ]:
!pip install rasterio
!pip install rio-cogeo
import rasterio

In [ ]:
from ee import oauth
from google_auth_oauthlib.flow import Flow
from pprint import pprint

# Download and process NLDAS data

In [ ]:
# Define time period for the downloading and processing task
years = ["2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019"]
months = ["01","02","03","04","05","06","07","08","09","10","11","12"]
days = ["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15",
        "16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

In [ ]:
# Define functions for automatic processing for different depth layers
def read_raster100(name, as_type=float): #100 cm depth
  r = rasterio.open(name)
  b = r.read(1)
  a = b.astype(as_type)
  return a
def average_rasters100(rasters):
  all_read_rasters = []
  total_input_rasters = len(rasters)
  for r in range(0, total_input_rasters):
        rr = read_raster100(rasters[r])
        all_read_rasters.append(rr)
  array_pixel_values = np.array(all_read_rasters)
  mean_array = np.nanmean(array_pixel_values, axis=0)
  temp = rasterio.open(rasters[0])
  with rasterio.open('mean_raster100.tif', 'w', driver='Gtiff',
                       width=temp.width, height=temp.height,
                       count=1, crs=temp.crs, transform=temp.transform,
                       dtype='float64') as mean_raster:
        mean_raster.write(mean_array, 1)
        mean_raster.close()
def read_raster50(name, as_type=float): #50 cm depth
  r = rasterio.open(name)
  b = r.read(2)
  a = b.astype(as_type)
  return a
def average_rasters50(rasters):
  all_read_rasters = []
  total_input_rasters = len(rasters)
  for r in range(0, total_input_rasters):
        rr = read_raster50(rasters[r])
        all_read_rasters.append(rr)
  array_pixel_values = np.array(all_read_rasters)
  mean_array = np.nanmean(array_pixel_values, axis=0)
  temp = rasterio.open(rasters[0])
  with rasterio.open('mean_raster50.tif', 'w', driver='Gtiff',
                       width=temp.width, height=temp.height,
                       count=1, crs=temp.crs, transform=temp.transform,
                       dtype='float64') as mean_raster:
        mean_raster.write(mean_array, 1)
        mean_raster.close()
def read_raster5(name, as_type=float): #5 cm depth
  r = rasterio.open(name)
  b = r.read(3)
  a = b.astype(as_type)
  return a
def average_rasters5(rasters):
  all_read_rasters = []
  total_input_rasters = len(rasters)
  for r in range(0, total_input_rasters):
        rr = read_raster5(rasters[r])
        all_read_rasters.append(rr)
  array_pixel_values = np.array(all_read_rasters)
  mean_array = np.nanmean(array_pixel_values, axis=0)
  temp = rasterio.open(rasters[0])
  with rasterio.open('mean_raster5.tif', 'w', driver='Gtiff',
                       width=temp.width, height=temp.height,
                       count=1, crs=temp.crs, transform=temp.transform,
                       dtype='float64') as mean_raster:
        mean_raster.write(mean_array, 1)
        mean_raster.close()
def read_raster25(name, as_type=float): #25 cm depth
  r = rasterio.open(name)
  b = r.read(4)
  a = b.astype(as_type)
  return a
def average_rasters25(rasters):
  all_read_rasters = []
  total_input_rasters = len(rasters)
  for r in range(0, total_input_rasters):
        rr = read_raster25(rasters[r])
        all_read_rasters.append(rr)
  array_pixel_values = np.array(all_read_rasters)
  mean_array = np.nanmean(array_pixel_values, axis=0)
  temp = rasterio.open(rasters[0])
  with rasterio.open('mean_raster25.tif', 'w', driver='Gtiff',
                       width=temp.width, height=temp.height,
                       count=1, crs=temp.crs, transform=temp.transform,
                       dtype='float64') as mean_raster:
        mean_raster.write(mean_array, 1)
        mean_raster.close()
def read_raster70(name, as_type=float): #70 cm depth
  r = rasterio.open(name)
  b = r.read(5)
  a = b.astype(as_type)
  return a
def average_rasters70(rasters):
  all_read_rasters = []
  total_input_rasters = len(rasters)
  for r in range(0, total_input_rasters):
        rr = read_raster70(rasters[r])
        all_read_rasters.append(rr)
  array_pixel_values = np.array(all_read_rasters)
  mean_array = np.nanmean(array_pixel_values, axis=0)
  temp = rasterio.open(rasters[0])
  with rasterio.open('mean_raster70.tif', 'w', driver='Gtiff',
                       width=temp.width, height=temp.height,
                       count=1, crs=temp.crs, transform=temp.transform,
                       dtype='float64') as mean_raster:
        mean_raster.write(mean_array, 1)
        mean_raster.close()

In [ ]:
# Processing all data and deposit to Cloud bucket
for year in years:
  for month in months:
    for day in days:
      try:
        date = year + month + day
        with open("NLDAS_download_list.txt") as f, open("urls.txt","w") as fout: # Customize the file name here based on how/where the file with a list of files to download is saved
          urls = re.findall(r'http*.*A' + date +'.*', f.read())
          fout.write("\n".join(urls))
        !wget --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --auth-no-challenge=on --keep-session-cookies --user=username --password=password --content-disposition -i urls.txt 
        # Customize the username and password here with the sign in information used for the Earthdata account for downloading NLDAS
        image_list = []
        for files in glob.glob("*SUB.tif"):
          image_list.append(files)
          image_name_nopath = os.path.basename(files)
        source_rasters = image_list
        average_rasters100(source_rasters)   
        average_rasters50(source_rasters) 
        average_rasters5(source_rasters) 
        average_rasters25(source_rasters) 
        average_rasters70(source_rasters)
        file_list = ['mean_raster5.tif','mean_raster25.tif','mean_raster50.tif','mean_raster70.tif','mean_raster100.tif']
        with rasterio.open(file_list[0]) as src0:
          meta = src0.meta
          meta.update(count = len(file_list))
        with rasterio.open('NLDAS_' + date +'.tif', 'w', **meta) as dst:
          for id, layer in enumerate(file_list, start=1):
            with rasterio.open(layer) as src1:
              dst.write_band(id, src1.read(1))
        !rm *SUB.tif
        !rm mean_raster*.tif
        filename = "NLDAS_" + date +".tif"
        !rio cogeo create $filename cog_$filename
        !gsutil cp cog_$filename gs://rangelands/Moisture/$year/"NLDAS_"$date".tif"
        !rm *.tif      
        !rm urls.txt 
      except IndexError:
        continue

# Upload dataset as a Google Earth Engine asset

In [ ]:
from google.auth.transport.requests import AuthorizedSession
ee.Authenticate()  
session = AuthorizedSession(ee.data.get_persistent_credentials())

In [ ]:
def requestBody(uris): # Customize the following digits to make sure date format is correct based on the file name 
  assetname = uris.split("/")[-1][0:-4]
  year = assetname[-8:-4]   
  month = assetname[-4:-2]  
  day = assetname[-2:]  
  date= year +"-"+month+"-"+day + "T00:00:00.000000000Z"
  print(date)
  request = {
    'type': 'IMAGE',
    'gcs_location': {
      'uris':[uris]
    },
    'startTime': date
  }
  pprint(json.dumps(request))
  return request

In [ ]:
def sendRequest(request,asset_id):
  project_folder = 'project_folder' # Customize the foldername here based on the project bucket
  url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'
  response = session.post(
    url = url.format(project_folder, asset_id),
    data = json.dumps(request)
  )
  pprint(json.loads(response.content))

In [ ]:
# Import all deposited files from the directory/ subdirectories from the Cloud bucket
files = !gsutil ls -r gs://rangelands/Moisture/**tif

In [ ]:
# Upload all NLDAS files saved in Cloud bucket to Google Earth Engine for quicker processing
for uris in files: 
  assetname= uris.split("/")[-1][0:-4] # Customize digits here based on file name
  assetpath="pathway"+assetname # Customize pathway here but make sure to first create the image collection in Google Earth Engine
  print(assetpath)
  thisrequest = requestBody(uris)
  sendRequest(thisrequest,assetpath)